In [1]:
from astropy.io import fits
import numpy as np
import os.path
import matplotlib.pyplot as plt
from astropy import wcs
from astropy.nddata import Cutout2D
from astropy import coordinates
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import matplotlib.gridspec as gridspec
import aplpy
import pandas as pd
from photutils import SkyCircularAperture
from photutils import aperture_photometry

In [2]:
path = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT/photz.zout'
#path = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT_extra/photz.zout'
cat = pd.read_csv(path, delim_whitespace=True, header=None, names=['ID', 'z_spec', 'z_a','f','fdf','l1','u1','l2','u2','l3','u3','3f','s2s','gt4','4f','bt'], skiprows=[0,1])

#getting z with +-
n=195
for i in range(len(cat)):
    if cat['ID'][i]==n:
        print('$z=' + str(cat['z_a'][i]) + '^{+' + str(cat['u1'][i]-cat['z_a'][i]) + '}_{-' + str(cat['z_a'][i]-cat['l1'][i]) + '}$')

#printing out an entire LateX final_table 
#(not big_table, which would be in the appendix and would be separated into several tables)
path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/final_table.csv'
final_table = pd.read_csv(path)
final_table1 = final_table[:55]
final_table2 = final_table[56:]
#print(final_table1.to_latex())
#print(final_table2.to_latex())

#print redshift test numbers here
path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/big_table.fits'
big_table = fits.open(path)[1].data
n_visual=np.zeros(9)
n_color=np.zeros(7)
n_stacking=np.zeros(9)
n_z=0
hrp=0
hrn=0
z5=0
z8=0
z4=0

for i in range(len(big_table)):
    if big_table.field('GOODS_z_Flag')[i]==1:
        n_z+=1
    n_visual[big_table.field('Visual_Flag')[i]-1] += 1
    n_color[big_table.field('Color_Color_Flag')[i]+1] += 1
    n_stacking[big_table.field('Stacking')[i] + 1] += 1
    if (big_table.field('Hardness_Ratio')[i] < 0) & (big_table.field('Visual_Flag')[i] > 3):
        hrn +=1
    if (big_table.field('Hardness_Ratio')[i] > 0) & (big_table.field('Visual_Flag')[i] > 3):
        hrp +=1
    if (big_table.field('phot_z')[i] > 5):
        if (big_table.field('phot_z')[i] < 8):
            z8 +=1
        else:
            z5 +=1
    elif (big_table.field('phot_z')[i] > 4):
        z4 +=1
    if big_table.field('Stacking')[i] == -1:
        print(i+1, big_table.field('BViz_Flag')[i], big_table.field('BVi_Flag')[i], big_table.field('BV_Flag')[i], big_table.field('GOODS_b_Flag')[i])
print(n_visual, n_color, n_stacking)
print(hrn, hrp, z5, z8, z4, n_z)

#print X-ray flux
i=928
print(big_table.field('Soft_Flux')[i], big_table.field('Hard_Flux')[i], big_table.field('Full_Flux')[i], big_table.field('Hardness_Ratio_2')[i])

#compare stack detections with individual goods bands detections
b=0
v=0
i=0
z=0
bv=0
bvi=0
bviz=0
for j in range(len(big_table)):
    if big_table.field('Visual_Flag')[j]>3:
        if big_table.field('GOODS_b_Flag')[j]==1:
            b +=1
        if big_table.field('GOODS_v_Flag')[j]==1:
            v +=1
        if big_table.field('GOODS_i_Flag')[j]==1:
            i +=1
        if big_table.field('GOODS_z_Flag')[j]==1:
            z +=1
        if big_table.field('BV_Flag')[j]==1:
            bv +=1
        if big_table.field('BVi_Flag')[j]==1:
            bvi +=1
        if big_table.field('BViz_Flag')[j]==1:
            bviz +=1
print(b*1.1, v*1.1, i*1.1, z*1.1, bv*1.1, bvi*1.1, bviz*1.1)

#calculate off-axis angle in arcmin, then angle survey with ^2*3.14
ra_aim=53.11691595
dec_aim=-27.80734248
path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/short_table.fits'
cat = fits.open(path)[1].data
survey_angle=0.
for i in range(len(cat)):
    survey_angle= (((((cat.field('RA')[i]-ra_aim)**2+(cat.field('DEC')[i]-dec_aim)**2)**0.5)*60.)**2)*3.14159
    print(survey_angle, (survey_angle/3.14159)**0.5)


$z=10.913^{+-0.242}_{-8.124}$


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:37: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:38: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


354 0.0 1.0 1.0 0
356 0.0 1.0 0.0 1
412 0.0 1.0 1.0 0
[ 455.  423.   18.   46.   18.   12.   16.   19.    1.] [   4.  964.    0.    0.    0.   25.   15.] [   3.  896.    0.    0.    0.   44.   12.   13.   40.]
48 64 4 6 4 69
3.7825910042e-16 5.77883196344e-16 9.5614229445e-16 -0.296941123213
7.700000000000001 52.800000000000004 61.60000000000001 75.9 50.6 64.9 75.9
32.0386560996 3.19346670417
35.8682260906 3.37893779289
110.650611851 5.93474628056
65.3690618832 4.56153879995
222.575222073 8.41712263474
104.593970843 5.7700366616
137.2102346 6.60873747775


In [3]:
#create tables for latex: 1-55 and 56 to 110:
#all the stuff I want to show:  8
# ID, RAX, DECX, h/s/f counts,errors,s/n 12
# ID, HR, HRu/l, HRsigma, Visual, Stack, color, zphot, photz1,2,3,u,l, chi2 14
#ID, RAH, DECH, flux and flux errors for all bands 13
#ID, magnitudes and mag errors for all bands 11
#

path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/big_table.fits'
big_table = fits.open(path)[1].data
path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/tbl04.fits'
cat7ms = fits.open(path)[1].data

def sci_notation(number, sig_fig=2):
    ret_string = '{0:.{1:d}e}'.format(number, sig_fig)
    a,b = ret_string.split('e')
    b = int(b) #removed leading "+" and strips leading zeros too.
    return a + ' $\times$ 10$^{' + str(b) + '}$'

mj=1000000

for ntables in [0,1]:
    
    flux_cat1a = [[],[],[],[],[],[],[],[],[],[],[],[]]
    flux_cat2a = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    flux_cat3a = [[],[],[],[],[],[],[],[],[],[],[],[],[]]
    flux_cat4a = [[],[],[],[],[],[],[],[],[],[],[]]
    flux_cat5a = [[],[],[],[],[],[],[],[],[],[],[]]
    flux_cat6a = [[],[],[],[],[],[],[],[],[],[],[]]

    for i in range(len(big_table)):
        if big_table.field('Visual_Flag')[i] > 3:
            if ((i < 524) & (ntables == 0)) | ((i>=524) & (ntables == 1)):
                if (i==526) | (i==621) | (i==639) | (i==713):
                    flux_cat1a[0].append('*' + str(big_table.field('ID')[i].astype('int64')))
                else:
                    flux_cat1a[0].append(big_table.field('ID')[i].astype('int64'))
                flux_cat1a[1].append(np.round(big_table.field('RA')[i], decimals=4))
                flux_cat1a[2].append(np.round(big_table.field('Dec')[i], decimals=4))
                flux_cat1a[3].append(np.round(cat7ms.field('FB_COUNTS')[i].astype('float64'), decimals=2))
                flux_cat1a[4].append(np.round(max(cat7ms.field('FB_LE')[i].astype('float64'), cat7ms.field('FB_UE')[i].astype('float64')), decimals=2))
                flux_cat1a[5].append(np.round(max(cat7ms.field('FB_COUNTS')[i].astype('float64')/max(cat7ms.field('FB_LE')[i].astype('float64'), cat7ms.field('FB_UE')[i].astype('float64')),-1.), decimals=2))
                flux_cat1a[6].append(np.round(cat7ms.field('SB_COUNTS')[i].astype('float64'), decimals=2))
                flux_cat1a[7].append(np.round(max(cat7ms.field('SB_LE')[i].astype('float64'), cat7ms.field('SB_UE')[i].astype('float64')), decimals=2))
                if big_table.field('SN_Ratio_Soft')[i] == 0:
                    flux_cat1a[8].append(np.round(-1.,decimals=2))
                else:
                    flux_cat1a[8].append(np.round(big_table.field('SN_Ratio_Soft')[i],decimals=2))
                flux_cat1a[9].append(np.round(cat7ms.field('HB_COUNTS')[i].astype('float64'),decimals=2))
                flux_cat1a[10].append(np.round(max(cat7ms.field('HB_LE')[i].astype('float64'), cat7ms.field('HB_UE')[i].astype('float64')), decimals=2))
                if big_table.field('SN_Ratio_Hard')[i] == 0:
                    flux_cat1a[11].append(np.round(-1., decimals=2))
                else:
                    flux_cat1a[11].append(np.round(big_table.field('SN_Ratio_Hard')[i], decimals=2))

                if (i==526) | (i==621) | (i==639) | (i==713):
                    flux_cat2a[0].append('*' + str(big_table.field('ID')[i].astype('int64')))
                else:
                    flux_cat2a[0].append(big_table.field('ID')[i].astype('int64'))
                if (big_table.field('Soft_Count_Err')[i] == -1) & (big_table.field('Hard_Count_Err')[i] == -1):
                    flux_cat2a[1].append('-')
                    flux_cat2a[2].append('-')
                    flux_cat2a[3].append('-')
                elif (big_table.field('Soft_Count_Err')[i] == -1):
                    flux_cat2a[1].append(str(np.round(big_table.field('Hardness_Ratio_2')[i],decimals=2)) + '$^l$')
                    flux_cat2a[2].append(str(np.round(big_table.field('Hardness_Ratio')[i],decimals=2)))
                    flux_cat2a[3].append('-')
                elif (big_table.field('Hard_Count_Err')[i] == -1):
                    flux_cat2a[1].append(str(np.round(big_table.field('Hardness_Ratio_2')[i],decimals=2)) + '$^u$')
                    flux_cat2a[2].append(str(np.round(big_table.field('Hardness_Ratio')[i],decimals=2)))
                    flux_cat2a[3].append('-')
                else:
                    flux_cat2a[1].append(str(np.round(big_table.field('Hardness_Ratio')[i],decimals=2)))
                    flux_cat2a[2].append('-')
                    flux_cat2a[3].append(str(np.round(big_table.field('Hardness_Ratio_Err')[i],decimals=2)))
                if big_table.field('Visual_Flag')[i] == 9.:
                    flux_cat2a[4].append(0)
                else:
                    flux_cat2a[4].append(np.round(big_table.field('Visual_Flag')[i]))
                if big_table.field('Stacking')[i] < 0:
                    flux_cat2a[5].append(0)
                else:
                    flux_cat2a[5].append(big_table.field('Stacking')[i].astype('int64'))
                if big_table.field('Color_Color_Flag')[i] < 0:
                    flux_cat2a[6].append(0)
                else:
                    flux_cat2a[6].append(big_table.field('Color_Color_Flag')[i].astype('int64'))
                flux_cat2a[7].append(np.round(big_table.field('Phot_z')[i],decimals=2))
                flux_cat2a[8].append(np.round(big_table.field('Phot_z_l1')[i],decimals=2))
                flux_cat2a[9].append(np.round(big_table.field('Phot_z_u1')[i],decimals=2))
                flux_cat2a[10].append(np.round(big_table.field('Phot_z_l2')[i],decimals=2))
                flux_cat2a[11].append(np.round(big_table.field('Phot_z_u2')[i],decimals=2))
                flux_cat2a[12].append(np.round(big_table.field('Phot_z_l3')[i],decimals=2))
                flux_cat2a[13].append(np.round(big_table.field('Phot_z_u3')[i],decimals=2))
                flux_cat2a[14].append(np.round(big_table.field('Chi_2')[i],decimals=2))

                if (i==526) | (i==621) | (i==639) | (i==713):
                    flux_cat3a[0].append('*' + str(big_table.field('ID')[i].astype('int64')))
                else:
                    flux_cat3a[0].append(big_table.field('ID')[i].astype('int64'))
                if (i==100) | (i==392) | (i==452) | (i==545):
                    flux_cat3a[1].append(np.round(big_table.field('HUGS_RA')[i],decimals=4))
                    flux_cat3a[2].append(np.round(big_table.field('HUGS_Dec')[i],decimals=4))
                else:    
                    flux_cat3a[1].append(np.round(big_table.field('CANDELS_160_RA')[i],decimals=4))
                    flux_cat3a[2].append(np.round(big_table.field('CANDELS_160_Dec')[i],decimals=4))
                if (big_table.field('GOODS_b_MagAper')[i] == 0):
                    flux_cat3a[3].append('-')
                else:
                    flux_cat3a[3].append(np.round(big_table.field('GOODS_b_MagAper')[i],decimals=3))
                if (big_table.field('GOODS_b_MagErrAper')[i] == 0):
                    flux_cat3a[4].append('-')
                else:
                    flux_cat3a[4].append(np.round(big_table.field('GOODS_b_MagErrAper')[i],decimals=3))
                if (big_table.field('GOODS_v_MagAper')[i] == 0):
                    flux_cat3a[5].append('-')
                else:
                    flux_cat3a[5].append(np.round(big_table.field('GOODS_v_MagAper')[i],decimals=3))
                if (big_table.field('GOODS_v_MagErrAper')[i] == 0):
                    flux_cat3a[6].append('-')
                else:
                    flux_cat3a[6].append(np.round(big_table.field('GOODS_v_MagErrAper')[i],decimals=3))
                if (big_table.field('GOODS_i_MagAper')[i] == 0):
                    flux_cat3a[7].append('-')
                else:
                    flux_cat3a[7].append(np.round(big_table.field('GOODS_i_MagAper')[i],decimals=3))
                if (big_table.field('GOODS_i_MagErrAper')[i] == 0):
                    flux_cat3a[8].append('-')
                else:
                    flux_cat3a[8].append(np.round(big_table.field('GOODS_i_MagErrAper')[i],decimals=3))
                if (big_table.field('GOODS_z_MagAper')[i] == 0):
                    flux_cat3a[9].append('-')
                else:
                    flux_cat3a[9].append(np.round(big_table.field('GOODS_z_MagAper')[i],decimals=3))
                if (big_table.field('GOODS_z_MagErrAper')[i] == 0):
                    flux_cat3a[10].append('-')
                else:
                    flux_cat3a[10].append(np.round(big_table.field('GOODS_z_MagErrAper')[i],decimals=3))
                if (big_table.field('CANDELS_105_MagAper')[i] == 0):
                    flux_cat3a[11].append('-')
                else:
                    flux_cat3a[11].append(np.round(big_table.field('CANDELS_105_MagAper')[i],decimals=3))
                if (big_table.field('CANDELS_105_MagErrAper')[i] == 0):
                    flux_cat3a[12].append('-')
                else:
                    flux_cat3a[12].append(np.round(big_table.field('CANDELS_105_MagErrAper')[i],decimals=3))

                if (i==526) | (i==621) | (i==639) | (i==713):
                    flux_cat4a[0].append('*' + str(big_table.field('ID')[i].astype('int64')))
                else:
                    flux_cat4a[0].append(big_table.field('ID')[i].astype('int64'))
                if (big_table.field('CANDELS_125_MagAper')[i] == 0):
                    flux_cat4a[1].append('-')
                else:
                    flux_cat4a[1].append(np.round(big_table.field('CANDELS_125_MagAper')[i],decimals=3))
                if (big_table.field('CANDELS_125_MagErrAper')[i] == 0):
                    flux_cat4a[2].append('-')
                else:
                    flux_cat4a[2].append(np.round(big_table.field('CANDELS_125_MagErrAper')[i],decimals=3))
                if (big_table.field('CANDELS_160_MagAper')[i] == 0):
                    flux_cat4a[3].append('-')
                else:
                    flux_cat4a[3].append(np.round(big_table.field('CANDELS_160_MagAper')[i],decimals=3))
                if (big_table.field('CANDELS_160_MagErrAper')[i] == 0):
                    flux_cat4a[4].append('-')
                else:
                    flux_cat4a[4].append(np.round(big_table.field('CANDELS_160_MagErrAper')[i],decimals=3))
                if (big_table.field('HUGS_MagAper')[i] == 0):
                    flux_cat4a[5].append('-')
                else:
                    flux_cat4a[5].append(np.round(big_table.field('HUGS_MagAper')[i],decimals=3))
                if (big_table.field('HUGS_MagErrAper')[i] == 0):
                    flux_cat4a[6].append('-')
                else:
                    flux_cat4a[6].append(np.round(big_table.field('HUGS_MagErrAper')[i],decimals=3))
                if (big_table.field('Spitzer_ch1_MagAper')[i] == 0):
                    flux_cat4a[7].append('-')
                else:
                    flux_cat4a[7].append(np.round(big_table.field('Spitzer_ch1_MagAper')[i],decimals=3))
                if (big_table.field('Spitzer_ch1_MagErrAper')[i] == 0):
                    flux_cat4a[8].append('-')
                else:
                    flux_cat4a[8].append(np.round(big_table.field('Spitzer_ch1_MagErrAper')[i],decimals=3))
                if (big_table.field('Spitzer_ch2_MagAper')[i] == 0):
                    flux_cat4a[9].append('-')
                else:
                    flux_cat4a[9].append(np.round(big_table.field('Spitzer_ch2_MagAper')[i],decimals=3))
                if (big_table.field('Spitzer_ch2_MagErrAper')[i] == 0):
                    flux_cat4a[10].append('-')
                else:
                    flux_cat4a[10].append(np.round(big_table.field('Spitzer_ch2_MagErrAper')[i],decimals=3))

                if (i==526) | (i==621) | (i==639) | (i==713):
                    flux_cat5a[0].append('*' + str(big_table.field('ID')[i].astype('int64')))
                else:
                    flux_cat5a[0].append(big_table.field('ID')[i].astype('int64'))
                if (big_table.field('GOODS_b_JanskyAper')[i] == 0):
                    flux_cat5a[1].append('-')
                else:
                    flux_cat5a[1].append(sci_notation(big_table.field('GOODS_b_JanskyAper')[i]*mj, sig_fig=3))
                if (big_table.field('GOODS_b_JanskyErrAper')[i] == 0):
                    flux_cat5a[2].append('-')
                else:
                    flux_cat5a[2].append(sci_notation(big_table.field('GOODS_b_JanskyErrAper')[i]*mj, sig_fig=3))
                if (big_table.field('GOODS_v_JanskyAper')[i] == 0):
                    flux_cat5a[3].append('-')
                else:
                    flux_cat5a[3].append(sci_notation(big_table.field('GOODS_v_JanskyAper')[i]*mj,sig_fig=3))
                if (big_table.field('GOODS_v_JanskyErrAper')[i] == 0):
                    flux_cat5a[4].append('-')
                else:
                    flux_cat5a[4].append(sci_notation(big_table.field('GOODS_v_JanskyErrAper')[i]*mj,sig_fig=3))
                if (big_table.field('GOODS_i_JanskyAper')[i] == 0):
                    flux_cat5a[5].append('-')
                else:
                    flux_cat5a[5].append(sci_notation(big_table.field('GOODS_i_JanskyAper')[i]*mj,sig_fig=3))
                if (big_table.field('GOODS_i_JanskyErrAper')[i] == 0):
                    flux_cat5a[6].append('-')
                else:
                    flux_cat5a[6].append(sci_notation(big_table.field('GOODS_i_JanskyErrAper')[i]*mj,sig_fig=3))
                if (big_table.field('GOODS_z_JanskyAper')[i] == 0):
                    flux_cat5a[7].append('-')
                else:
                    flux_cat5a[7].append(sci_notation(big_table.field('GOODS_z_JanskyAper')[i]*mj,sig_fig=3))
                if (big_table.field('GOODS_z_JanskyErrAper')[i] == 0):
                    flux_cat5a[8].append('-')
                else:
                    flux_cat5a[8].append(sci_notation(big_table.field('GOODS_z_JanskyErrAper')[i]*mj,sig_fig=3))
                if (big_table.field('CANDELS_105_JanskyAper')[i] == 0):
                    flux_cat5a[9].append('-')
                else:
                    flux_cat5a[9].append(sci_notation(big_table.field('CANDELS_105_JanskyAper')[i]*mj,sig_fig=3))
                if (big_table.field('CANDELS_105_JanskyErrAper')[i] == 0):
                    flux_cat5a[10].append('-')
                else:
                    flux_cat5a[10].append(sci_notation(big_table.field('CANDELS_105_JanskyErrAper')[i]*mj,sig_fig=3))

                if (i==526) | (i==621) | (i==639) | (i==713):
                    flux_cat6a[0].append('*' + str(big_table.field('ID')[i].astype('int64')))
                else:
                    flux_cat6a[0].append(big_table.field('ID')[i].astype('int64'))
                if (big_table.field('CANDELS_125_JanskyAper')[i] == 0):
                    flux_cat6a[1].append('-')
                else:
                    flux_cat6a[1].append(sci_notation(big_table.field('CANDELS_125_JanskyAper')[i]*mj,sig_fig=3))
                if (big_table.field('CANDELS_125_JanskyErrAper')[i] == 0):
                    flux_cat6a[2].append('-')
                else:
                    flux_cat6a[2].append(sci_notation(big_table.field('CANDELS_125_JanskyErrAper')[i]*mj,sig_fig=3))
                if (big_table.field('CANDELS_160_JanskyAper')[i] == 0):
                    flux_cat6a[3].append('-')
                else:
                    flux_cat6a[3].append(sci_notation(big_table.field('CANDELS_160_JanskyAper')[i]*mj,sig_fig=3))
                if (big_table.field('CANDELS_160_JanskyErrAper')[i] == 0):
                    flux_cat6a[4].append('-')
                else:
                    flux_cat6a[4].append(sci_notation(big_table.field('CANDELS_160_JanskyErrAper')[i]*mj,sig_fig=3))
                if (big_table.field('HUGS_JanskyAper')[i] == 0):
                    flux_cat6a[5].append('-')
                else:
                    flux_cat6a[5].append(sci_notation(big_table.field('HUGS_JanskyAper')[i]*mj,sig_fig=3))
                if (big_table.field('HUGS_JanskyErrAper')[i] == 0):
                    flux_cat6a[6].append('-')
                else:
                    flux_cat6a[6].append(sci_notation(big_table.field('HUGS_JanskyErrAper')[i]*mj,sig_fig=3))
                if (big_table.field('Spitzer_ch1_JanskyAper')[i] == 0):
                    flux_cat6a[7].append('-')
                else:
                    flux_cat6a[7].append(sci_notation(big_table.field('Spitzer_ch1_JanskyAper')[i]*mj,sig_fig=3))
                if (big_table.field('Spitzer_ch1_JanskyErrAper')[i] == 0):
                    flux_cat6a[8].append('-')
                else:
                    flux_cat6a[8].append(sci_notation(big_table.field('Spitzer_ch1_JanskyErrAper')[i]*mj,sig_fig=3))
                if (big_table.field('Spitzer_ch2_JanskyAper')[i] == 0):
                    flux_cat6a[9].append('-')
                else:
                    flux_cat6a[9].append(sci_notation(big_table.field('Spitzer_ch2_JanskyAper')[i]*mj,sig_fig=3))
                if (big_table.field('Spitzer_ch2_JanskyErrAper')[i] == 0):
                    flux_cat6a[10].append('-')
                else:
                    flux_cat6a[10].append(sci_notation(big_table.field('Spitzer_ch2_JanskyErrAper')[i]*mj,sig_fig=3))

    ar = np.array(flux_cat1a)
    at = np.transpose(ar)
    df1a = pd.DataFrame(data=at, index=None, columns=['ID','RA$_X$','Dec$_X$','FB\_Count','FB\_$\sigma$','FB\_S/N','SB\_Count','SB\_$\sigma$','SB\_S/N','FB\_Count','HB\_$\sigma$','HB\_S/N'])
    #df1a['ID'] = df1a['ID'].astype('int64')
    #df1a['RA$_X$'] = np.round(df1a['RA$_X$'], decimals=4) #actually do this already in the step above, so we can mix
    #strings and floats in the same column!
    #df1a['Dec$_X$'] = np.round(df1a['Dec$_X$'], decimals=4)
    #df1a['FB\_Count'] = np.round(df1a['FB\_Count'], decimals=2)
    #df1a['FB\_$\sigma$'] = np.round(df1a['FB\_$\sigma$'], decimals=2)
    #df1a['FB\_S/N'] = np.round(df1a['FB\_S/N'], decimals=2)


    ar = np.array(flux_cat2a)
    at = np.transpose(ar)
    df2a = pd.DataFrame(data=at, index=None, columns=['ID','HR','HR\_best','HR\_$\sigma$','Visual','Stacking','Colour','$z_{phot}$','$z_{l1}$','$z_{u1}$','$z_{l2}$','$z_{u2}$','$z_{l3}$','$z_{u3}$','$\chi^2$'])
    #df2a['ID'] = df2a['ID'].astype('int64')



    ar = np.array(flux_cat3a)
    at = np.transpose(ar)
    df3a = pd.DataFrame(data=at, index=None, columns=['ID','RA$_{ref}$','Dec$_{ref}$','$m(B)$','$merr(B)$','$m(V)$','$merr(V)$','$m(i)$','$merr(i)$','$m(z)$','$merr(z)$','$m(Y)$','$merr(Y)$'])
    #df3a['ID'] = df3a['ID'].astype('int64')



    ar = np.array(flux_cat4a)
    at = np.transpose(ar)
    df4a = pd.DataFrame(data=at, index=None, columns=['ID','$m(J)$','$merr(J)$','$m(H)$','$merr(H)$','$m(K_S)$','$merr(K_S)$','$m(3.6\mu)$','$merr(3.6\mu)$','$m(4.5\mu)$','$merr(4.5\mu)$'])
    #df4a['ID'] = df4a['ID'].astype('int64')



    ar = np.array(flux_cat5a)
    at = np.transpose(ar)
    df5a = pd.DataFrame(data=at, index=None, columns=['ID','$F_{\nu}(B)$','$Ferr_{\nu}(B)$','$F_{\nu}(V)$','$Ferr_{\nu}(V)$','$F_{\nu}(i)$','$Ferr_{\nu}(i)$','$F_{\nu}(z)$','$Ferr_{\nu}(z)$','$F_{\nu}(Y)$','$Ferr_{\nu}(Y)$'])
    #df5a['ID'] = df5a['ID'].astype('int64')



    ar = np.array(flux_cat6a)
    at = np.transpose(ar)
    df6a = pd.DataFrame(data=at, index=None, columns=['ID','$F_{\nu}(J)$','$Ferr_{\nu}(J)$','$F_{\nu}(H)$','$Ferr_{\nu}(H)$','$F_{\nu}(K_S)$','$Ferr_{\nu}(K_S)$','$F_{\nu}(3.6\mu)$','$Ferr_{\nu}(3.6\mu)$','$F_{\nu}(4.5\mu)$','$Ferr_{\nu}(4.5\mu)$'])
    #df6a['ID'] = df6a['ID'].astype('int64')



    print(df1a.to_latex(escape=False, index=False))
    print(df2a.to_latex(escape=False, index=False))
    print(df3a.to_latex(escape=False, index=False))
    print(df4a.to_latex(escape=False, index=False))
    print(df5a.to_latex(escape=False, index=False))
    print(df6a.to_latex(escape=False, index=False))

\begin{tabular}{rrrrrrrrrrrr}
\toprule
    ID &   RA$_X$ &  Dec$_X$ &  FB\_Count &  FB\_$\sigma$ &  FB\_S/N &  SB\_Count &  SB\_$\sigma$ &  SB\_S/N &  FB\_Count &  HB\_$\sigma$ &  HB\_S/N \\
\midrule
 101.0 &  53.0065 & -27.7341 &     185.42 &         28.45 &     6.52 &      55.92 &         15.60 &     3.59 &     129.62 &         24.38 &     5.32 \\
 135.0 &  53.0223 & -27.7789 &     617.91 &         30.84 &    20.04 &     161.38 &         16.02 &    10.07 &     457.12 &         26.89 &    17.00 \\
 150.0 &  53.0269 & -27.7652 &      68.25 &         17.10 &     3.99 &      24.01 &         -1.00 &    -1.00 &      55.51 &         14.94 &     3.71 \\
 185.0 &  53.0392 & -27.7994 &      49.63 &         12.38 &     4.01 &      21.44 &          7.70 &     2.79 &      28.29 &         10.51 &     2.69 \\
 189.0 &  53.0400 & -27.7984 &      50.93 &         12.90 &     3.95 &      16.45 &         -1.00 &    -1.00 &      42.55 &         11.37 &     3.74 \\
 194.0 &  53.0409 & -27.8360 &      67.0

In [4]:
for i in [526,621,639,713]:
    print(big_table.field('SN_Ratio_Soft')[i])

3.20164986631
2.78657811374
3.33958649563
4.72578086462
